<a href="https://colab.research.google.com/github/sulaimanbehzad/Clustering_according_to_item_popularity/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering of Digikala dataset based on items popularity  


In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize

### Cloning the github repository in colab

In [74]:
% cd /content/drive/MyDrive/Clustering_according_to_item_popularity/


/content/drive/MyDrive/Clustering_according_to_item_popularity


In [75]:
# ! ls

In [76]:
# ! git clone https://github.com/sulaimanbehzad/Clustering_according_to_item_popularity.git

In [77]:
! git status

On branch main
Your branch and 'origin/main' have diverged,
and have 2 and 4 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Clustering_according_to_item_popularity/
	preprocessed_item_recomm.csv

nothing added to commit but untracked files present (use "git add" to track)


In [78]:
! git pull


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ab657937bb61.(none)')


In [79]:
# ! git push -u origin main

In [80]:
# ! git commit -m 'redundant files removed'

### Reading and preprocessing the dataset

In [81]:
df = pd.read_excel('/content/drive/MyDrive/Clustering_according_to_item_popularity/keifiat.xlsx')

In [82]:
df.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [83]:
df.describe()

,product_id,user_id,likes,dislikes
count,100000.000000,1.000000e+05,100000.000000,100000.000000
mean,426482.510140,4.642959e+06,3.114790,1.649460
std,232470.511159,2.619034e+06,8.050031,8.912705
min,300.000000,4.661320e+05,0.000000,0.000000
25%,220871.250000,2.613059e+06,0.000000,0.000000
50%,427858.000000,4.710290e+06,1.000000,0.000000
75%,619397.250000,7.094806e+06,4.000000,1.000000
max,909423.000000,8.623288e+06,854.000000,1344.000000


Select the attributes that will give information about the popularity of the item

In [84]:
df1 = df[['product_id', 'likes', 'dislikes','comment', 'recommend']]

Check the dataframe for null values and handle them

In [85]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   product_id  100000 non-null  int64 
 1   likes       100000 non-null  int64 
 2   dislikes    100000 non-null  int64 
 3   comment     99883 non-null   object
 4   recommend   100000 non-null  object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


In [86]:
df1.isnull().sum()

product_id      0
likes           0
dislikes        0
comment       117
recommend       0
dtype: int64

looking at the database 'recommend' we can see that \N values also represent null values which wouldn't be considered in the above

In [87]:
df1['recommend'].value_counts()

recommended        36972
\N                 36382
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64

we can see that beside the 117 missing values we have 36382 records with \N values  
now we understand tha \N too should be considered as missing value


In [88]:
missing_values = ["\\N", "na", "--", "NA", "n/a"]
df = pd.read_excel('/content/drive/MyDrive/Clustering_according_to_item_popularity/keifiat.xlsx', na_values = missing_values)
df1 = df[['product_id', 'likes', 'dislikes','comment', 'recommend']]


after importing database with newly found aspects on NA we are going to handle null values

In [89]:
df1.isnull().sum()

product_id        0
likes             0
dislikes          0
comment         118
recommend     36382
dtype: int64

In [90]:
df1['recommend'].value_counts()

recommended        36972
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64

In [91]:
most_common = df1["recommend"].value_counts().idxmax
# df1['recommend'].fillna(median, inplace=True)
most_common

<bound method Series.idxmax of recommended        36972
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64>

replacing the null values with one of the above would falsely impact the clustering so let's take another approach:  
1. use interpolation to find values
2. fill with mean/median

in order to use interpolation let us first use label encoder to obtain numerical values

In [92]:
df1['recommend'].fillna(value='null_values', inplace=True)
df1.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


product_id      0
likes           0
dislikes        0
comment       118
recommend       0
dtype: int64

In [93]:
le = LabelEncoder()
le.fit(df1['recommend'])
le.classes_

array(['no_idea', 'not_recommended', 'null_values', 'recommended'],
      dtype=object)

In [94]:
recomm_trans = le.transform(df1['recommend'])
recomm_trans

array([2, 3, 1, ..., 3, 0, 0])

In [95]:
df1['recommend'] = recomm_trans

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
df1[['recommend']]

,recommend
0,2
1,3
2,1
3,0
4,0
...,...
99995,3
99996,2
99997,3
99998,0


In [97]:
df1['recommend'] = df1['recommend'].replace(2,np.nan)
df1['recommend']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0        NaN
1        3.0
2        1.0
3        0.0
4        0.0
        ... 
99995    3.0
99996    NaN
99997    3.0
99998    0.0
99999    0.0
Name: recommend, Length: 100000, dtype: float64

In [98]:
recomm_filled = df1['recommend'].interpolate(method='linear')
recomm_filled.isnull().sum()

1

In [99]:
df1['recommend'] = recomm_filled
df1.head(50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,product_id,likes,dislikes,comment,recommend
0,3692,0,0,واقعا عالیه. من که ازش خیلی راضیم,NaN
1,90213,4,1,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,3.000000
2,59473,1,0,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,1.000000
3,120499,6,11,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,0.000000
4,67200,19,4,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,0.000000
5,133722,6,1,سلام دوستان،،\r\nمنم مثه بعضی از دوستان قبل از...,3.000000
6,148509,3,0,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,3.000000
7,56871,2,2,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,2.000000
8,49738,2,1,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,1.000000
9,161548,1,0,خوب بودممنون,0.000000


In [100]:
df1.isnull().sum()

product_id      0
likes           0
dislikes        0
comment       118
recommend       1
dtype: int64

In [101]:
df1.dropna(inplace=True)
df1.isnull().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


product_id    0
likes         0
dislikes      0
comment       0
recommend     0
dtype: int64

## handling NULL values is DONE!!! 

next step is to drop duplicates

In [102]:
df1.drop_duplicates(subset='product_id', inplace=True, keep='first')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50189 entries, 1 to 99998
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  50189 non-null  int64  
 1   likes       50189 non-null  int64  
 2   dislikes    50189 non-null  int64  
 3   comment     50189 non-null  object 
 4   recommend   50189 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 2.3+ MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


More than 49000 products with duplicate records were deleted as a result
let's save the preproccessed dataframe upto now

In [103]:
# df1.to_csv('preprocessed_item_recomm.csv',index=False, encoding='utf-8-sig')
prep_df.head(50)

,product_id,likes,dislikes,comment,recommend
0,90213,4,1,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,3.000000
1,59473,1,0,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,1.000000
2,120499,6,11,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,0.000000
3,67200,19,4,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,0.000000
4,133722,6,1,سلام دوستان،،\r\nمنم مثه بعضی از دوستان قبل از...,3.000000
5,148509,3,0,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,3.000000
6,56871,2,2,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,2.000000
7,49738,2,1,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,1.000000
8,161548,1,0,خوب بودممنون,0.000000
9,5107,1,0,نزدیک به دو ساله که این هارد رو خریدم. استفاده...,3.000000


In [108]:
prep_df = pd.read_csv('/content/drive/MyDrive/Clustering_according_to_item_popularity/preprocessed_item_recomm.csv')
prep_df['recommend']=prep_df['recommend'].round()
prep_df['recommend'] = pd.to_numeric(prep_df['recommend'], downcast='integer')
inv_tr=le.inverse_transform(prep_df['recommend'])
inv_tr

array(['recommended', 'not_recommended', 'no_idea', ..., 'recommended',
       'recommended', 'no_idea'], dtype=object)

In [109]:
dummies = pd.get_dummies(prep_df['recommend'])
dummies.head()

,0,1,2,3
0,0,0,0,1
1,0,1,0,0
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1
